In [152]:
# Importing necessary modules

import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil

In [153]:
# Identifying path for mass-data retrieval

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)

all_datasheets = glob.glob(datasheets + "/*.csv")

In [242]:
for sheet in all_datasheets:
    lab_data = pd.read_csv(sheet)
    
    # Checking for data - skipping remainder of code (breaking loop) if there is no data
    
    if np.isnan(lab_data.Absorbance[0]):
        break
    
    # Continuing if data is found
    
    else:
        print(sheet)
        
        
        # Identifying standard absorbances based on 'Type' column, and adding to list of values to plot
        
        curve_absorbance = []
        curve_concentration = []
        
        for i in range(len(lab_data)):
            if lab_data.Type[i] == 'C':
                curve_absorbance.append(int(lab_data.Absorbance[i]))
                curve_concentration.append(int(lab_data.Sample_ID[i]))
  
    
    # calculating r^2 value for two lists of values
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(curve_absorbance, curve_concentration)
    print(r_value**2)
    
    # creating conditional for screening based on r^2 value
    
    if r_value**2 < 0.8:
        
        # Moving bad curve sheet to new folder to avoid running again in the future
        
        sheet_name = sheet.split('\\')[-1]
        shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Questionable_Data/BADCURVE_' + str(sheet_name))
        
        # Creating new blank dataframe for running samples again
        
        sheet_redo = lab_data
        sheet_redo['Sample_Wt(g)'] = ''
        sheet_redo['Absorbance'] = ''
        
        # Reassigning navigation variables to save ouput files in a new location
            
        folder = "Lab_Sheets"

        save_loc = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

            
        # Navigating to specified saving location
            
        glob.os.chdir(save_loc)
        
        
        # Saving new dataframe in Lab_Sheets folder with 'B' to indicate re-running samples
        
        sheet
        sheet_redo.to_csv('B_' + '_'.join(sheet_name.split('_')[1:]), index=False)
    
    
    # if the r^2 value is acceptable, look at variation between samples...
    
    else: 
        
        # define which rows contain data for samples, and creating new df with only absorbance
        
        sample_data = lab_data[lab_data.Type != 'C']
        sample_data = sample_data[sample_data.Type != 'B']
        sample_data_ab = sample_data[['Sample_ID', 'Absorbance']]
        
        
        # group sample data based on Sample_ID information
        
        sample_data_g = sample_data_ab.groupby('Sample_ID')
        
        
        # merge and create new dataframe for variance values
        
        sample_data_var = sample_data_ab.merge((abs((sample_data_g.max() - sample_data_g.min())) / (sample_data_g.max())), on='Sample_ID')
        
        # Create list of sample ID that should be re-run
        
        Re_Run = []
        
        for i in range(len(lab_data)):
            if lab_data.Type[i] == 'C':
                Re_Run.append(lab_data.Sample_ID)
        
        # create conditional based on variance column
        
        for i, row in sample_data_var.iterrows():
            if float(row['Absorbance_y']) > 0.1:
                Re_Run.append(row['Sample_ID'])
                
        # create a conditional based on the presence of samples that need to be run again
        
        if Re_Run != []:
            sample_redo = lab_data
            sample_redo['Sample_Wt(g)'] = ''
            sample_redo['Absorbance'] = ''
            sample_redo['Sample_ID'] = ''
            sample_redo['Sample_ID'] = Re_Run
            
            ### CREATING NEW CSV FILE WITH SAMPLES AND STANDARDS TO RUN ###
            
            ### ... this could just be a list of samples saved to STUDENT_LIST folder #####
            # adding curve values is not necessary ... will need to loop for all files, and
            # be based on ureides/nitrates
        
        print(sample_redo)

/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets\A_1_NIT_MR_USB_00_00_00 (good curve).csv
1.0


ValueError: Length of values does not match length of index

In [133]:
'B_'+'_'.join(sheet_name.split('_')[1:])

'B_1_NIT_RV_CTS_00_00_00 (bad curve).csv'

In [241]:
for sheet in all_datasheets:
    lab_data = pd.read_csv(sheet)
    
    # Checking for data - skipping remainder of code (breaking loop) if there is no data
    
    if np.isnan(lab_data.Absorbance[0]):
        break
    
    # Continuing if data is found
    
    else:
        print(sheet)
        
        
        # Identifying standard absorbances based on 'Type' column, and adding to list of values to plot
        
        curve_absorbance = []
        curve_concentration = []
        
        for i in range(len(lab_data)):
            if lab_data.Type[i] == 'C':
                curve_absorbance.append(int(lab_data.Absorbance[i]))
                curve_concentration.append(int(lab_data.Sample_ID[i]))
                
                
    # taking list of values, plotting, and fitting line (probably not necessary to show)            
        
    #plt.scatter(curve_absorbance, curve_concentration)
    #z = np.polyfit(curve_concentration, curve_absorbance, 1)
    #p = np.poly1d(z)
    #plt.plot(curve_concentration, p(curve_concentration),"g--")
    #plt.show()
    
    
    # calculating r^2 value for two lists of values
    
    slope, intercept, r_value, p_value, std_err = stats.linregress(curve_absorbance, curve_concentration)
    print(r_value**2)
    
    # creating conditional for screening based on r^2 value
    
    if r_value**2 < 0.8:
        print("Bad Curve")
        
    # if the r^2 value is acceptable...

    
    
### NEW CODE ###

    else: 
        
        # define which rows contain data for samples, and creating new df with only absorbance
        
        sample_data = lab_data[lab_data.Type != 'C']
        sample_data = sample_data[sample_data.Type != 'B']
        sample_data_ab = sample_data[['Sample_ID', 'Absorbance']]
        
        
        # group sample data based on Sample_ID information
        
        sample_data_g = sample_data_ab.groupby('Sample_ID')
        
        
        # merge and create new dataframe for variance values
        
        sample_data_var = sample_data_ab.merge((abs((sample_data_g.max() - sample_data_g.min())) / (sample_data_g.max())), on='Sample_ID')
        
        # Create list of sample ID that should be re-run
        
        Re_Run = []
        
        for i in range(len(lab_data)):
            if lab_data.Type[i] == 'C':
                Re_Run.append(lab_data.Sample_ID)
        
        # create conditional based on variance column
        
        for i, row in sample_data_var.iterrows():
            if float(row['Absorbance_y']) > 0.1:
                Re_Run.append(row['Sample_ID'])
                
        # create a conditional based on the presence of samples that need to be run again
        
        if Re_Run != []:
            sample_redo = lab_data
            sample_redo['Sample_Wt(g)'] = ''
            sample_redo['Absorbance'] = ''
            sample_redo['Sample_ID'] = ''
            sample_redo['Sample_ID'] = Re_Run
            
        
        print(sample_redo)

/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets\A_1_NIT_MR_USB_00_00_00 (good curve).csv
1.0


ValueError: Length of values does not match length of index